In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge, BayesianRidge
from sklearn.datasets import make_regression

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [13]:
team_data = pd.read_csv('../Stats_competition-/final_opponent_and_team_data.csv')
team_data['Location'] = np.where(team_data['Location'] == 'N', 0, np.where(team_data['Location'] == 'H', 1, -1))
columns_to_convert = ['Location','ADJO', 'ADJD', 'EFG%', 'TO%', 'OR%', 'FTR', 'Opp EFG%', 'Opp TO%', 'Opp OR%', 'Opp FTR']
for col in columns_to_convert:
    team_data[col] = pd.to_numeric(team_data[col], errors='coerce')
    
cleanDate = team_data.dropna()

In [14]:
# Removing columns not needed for predictions
feature_cols = cleanDate.columns.difference(['Date', 'Team', 'Opponent', 'Team_score', 'Opponent_score',
                                             '2P', '3P', 'Opp 2P', 'Opp 3P',
                                             'EFF', 'Opp EFF', 'WAB'])

# Defining the feature matrix (X) and target matrix (y)
X = cleanDate[feature_cols]
y = cleanDate[['Team_score', 'Opponent_score']]

display(X.head())
display(y.head())

,ADJD,ADJO,EFG%,FTR,Location,OR%,Opp EFG%,Opp FTR,Opp OR%,Opp TO%,TO%,opp_adj_d,opp_adj_o
0,95.2,125.3,64.3,31.7,1,35.5,39.5,31.6,19.4,19.0,17.7,110.8,108.9
1,92.3,124.7,61.3,25.4,1,43.6,39.1,14.1,23.3,18.3,11.3,119.0,107.2
2,86.4,106.8,42.3,23.9,0,25.0,47.6,38.1,23.3,14.6,9.3,88.1,119.6
3,56.9,124.7,61.3,17.7,1,45.2,28.9,5.3,29.3,29.5,15.5,115.5,69.8
4,75.9,111.7,50.0,21.3,-1,35.1,45.3,20.8,16.7,22.2,20.7,90.4,98.9


,Team_score,Opponent_score
0,96,62
1,100,58
2,72,77
3,86,35
4,69,55


In [15]:
display(X.head())
display(y.head())

,ADJD,ADJO,EFG%,FTR,Location,OR%,Opp EFG%,Opp FTR,Opp OR%,Opp TO%,TO%,opp_adj_d,opp_adj_o
0,95.2,125.3,64.3,31.7,1,35.5,39.5,31.6,19.4,19.0,17.7,110.8,108.9
1,92.3,124.7,61.3,25.4,1,43.6,39.1,14.1,23.3,18.3,11.3,119.0,107.2
2,86.4,106.8,42.3,23.9,0,25.0,47.6,38.1,23.3,14.6,9.3,88.1,119.6
3,56.9,124.7,61.3,17.7,1,45.2,28.9,5.3,29.3,29.5,15.5,115.5,69.8
4,75.9,111.7,50.0,21.3,-1,35.1,45.3,20.8,16.7,22.2,20.7,90.4,98.9


,Team_score,Opponent_score
0,96,62
1,100,58
2,72,77
3,86,35
4,69,55


## Train Model

In [16]:
## Train Bayesian Ridge regression models for multiple target variables without evaluation.
def train_BaseModel(X, y):
    models = {}
    for target in y.columns:
        print(f"Training model for {target}...")

        # Initialize the model
        model = BayesianRidge()
        
        # Train the model
        model.fit(X, y[target])
        models[target] = model

        print(f"Model for {target} trained successfully.\n")

    return models

# Train a StackingRegressor for multiple target variables without evaluation.
def train_EnsembleModel(X, y, threshold=6):
    models = {}
    for target in y.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('ridge', Ridge()),
            ('bayesian_ridge', BayesianRidge())
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X, y[target])
        models[target] = stacking_model

        print(f"Model for {target} trained successfully.\n")

    return models


# Train a StackingRegressor combining Bayesian Ridge and Random Forest for multiple target variables.
def train_EnsembleModelWithRF(X_train, y_train, threshold=6):
    models = {}
    for target in y_train.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('bayesian_ridge', BayesianRidge()),
            ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X_train, y_train[target])
        models[target] = stacking_model

        print(f"Model for {target} trained successfully.\n")

    return models

# Train Bayesian Ridge regression models with hyperparameter tuning for multiple target variables.
def train_baseModelWithHyperparameterTuning(X_train, y_train, threshold=6):
    models = {}

    for target in y_train.columns:
        print(f"Training model for {target}...\n")

        # Step 1: Broad parameter ranges for RandomizedSearchCV
        random_param_grid = {
            'alpha_1': np.logspace(-6, -2, 20),  # Wide range
            'alpha_2': np.logspace(-6, -2, 20),  # Wide range
            'lambda_1': np.logspace(-6, -2, 20), # Wide range
            'lambda_2': np.logspace(-6, -2, 20), # Wide range
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }

        model = BayesianRidge()
        scorer = 'neg_mean_squared_error'

        random_search = RandomizedSearchCV(
            model, random_param_grid, scoring=scorer, cv=5, n_iter=50, verbose=1, random_state=42
        )
        random_search.fit(X_train, y_train[target])
        best_params_random = random_search.best_params_
        print(f"Best parameters from RandomizedSearchCV for {target}:", best_params_random)

        # Step 2: Define a narrower grid around the best parameters
        grid_param_grid = {
            'alpha_1': np.linspace(
                best_params_random['alpha_1'] / 10, best_params_random['alpha_1'] * 10, 10
            ),
            'alpha_2': np.linspace(
                best_params_random['alpha_2'] / 10, best_params_random['alpha_2'] * 10, 10
            ),
            'lambda_1': np.linspace(
                best_params_random['lambda_1'] / 10, best_params_random['lambda_1'] * 10, 10
            ),
            'lambda_2': np.linspace(
                best_params_random['lambda_2'] / 10, best_params_random['lambda_2'] * 10, 10
            ),
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }

        grid_search = GridSearchCV(model, grid_param_grid, scoring=scorer, cv=5, verbose=1)
        grid_search.fit(X_train, y_train[target])
        best_params_grid = grid_search.best_params_
        print(f"Best parameters from GridSearchCV for {target}:", best_params_grid)

        # Step 3: Train the model with the best parameters
        final_model = BayesianRidge(**best_params_grid)
        final_model.fit(X_train, y_train[target])
        models[target] = final_model

        print(f"Model for {target} trained successfully.\n")

    return models

model1 = train_BaseModel(X, y)
print(model1)

model2 = train_EnsembleModel(X, y)
print(model2)

model3 = train_EnsembleModelWithRF(X, y)
print(model3)

model4 = train_baseModelWithHyperparameterTuning(X, y)
print(model4)


Training model for Team_score...
Model for Team_score trained successfully.

Training model for Opponent_score...
Model for Opponent_score trained successfully.

{'Team_score': BayesianRidge(), 'Opponent_score': BayesianRidge()}
Training StackingRegressor model for Team_score...
Model for Team_score trained successfully.

Training StackingRegressor model for Opponent_score...
Model for Opponent_score trained successfully.

{'Team_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge()), 'Opponent_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge())}
Training StackingRegressor model for Team_score...
Model for Team_score trained successfully.

Training StackingRegressor model for Opponent_score...
Model for Opponent_score trained successfully.

{'

## Test model using test data

In [17]:
def predictModel(models, data):
    # Preprocess the data
    print("Preprocessing data for prediction...")
    data['Location'] = np.where(data['Location'] == 'Neutral', 0, 
                                np.where(data['Location'] == 'Home', 1, -1))
    
    columns_to_convert = ['ADJD', 'ADJO', 'EFG%', 'FTR', 'Location', 'OR%', 
                        'Opp EFG%', 'Opp FTR', 'Opp OR%', 'Opp TO%', 'TO%',
                        'opp_adj_d', 'opp_adj_o']
    
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    
    
    # Extract the features for prediction
    X = data[columns_to_convert]

    # Make predictions using the models
    print("Making predictions...")
    predictions = pd.DataFrame()
    for target, model in models.items():
        print(f"Predicting {target}...")
        predictions[target] = model.predict(X)

    print("Predictions completed.\n")
    
    teams = data[['Team', 'Opponent']]
    finalPredictions = pd.concat([teams, predictions], axis=1)
    
    return finalPredictions

predict_data = pd.read_csv('../Stats_competition-/basketball_games_data.csv')
predict1 = predictModel(model1, predict_data)
predict2 = predictModel(model2, predict_data)
predict3 = predictModel(model3, predict_data)
predict4 = predictModel(model4, predict_data)

predict1 = predict1.rename(columns={'Team_score': 'Team_score_base', 'Opponent_score': 'Opponent_score_base'})
predict2 = predict2.rename(columns={'Team_score': 'Team_score_ensemble', 'Opponent_score': 'Opponent_score_ensemble'})
predict3 = predict3.rename(columns={'Team_score': 'Team_score_ensembleRF', 'Opponent_score': 'Opponent_score_ensembleRF'})
predict4 = predict4.rename(columns={'Team_score': 'Team_score_fineTuned', 'Opponent_score': 'Opponent_score_fineTuned'})

# Join all four DataFrames on 'team' and 'opponent'
dfFinalPrediction1 = predict1.merge(predict2, on=['Team', 'Opponent'], how='outer')
dfFinalPrediction2 = predict3.merge(predict4, on=['Team', 'Opponent'], how='outer')

Preprocessing data for prediction...
Making predictions...
Predicting Team_score...
Predicting Opponent_score...
Predictions completed.

Preprocessing data for prediction...
Making predictions...
Predicting Team_score...
Predicting Opponent_score...
Predictions completed.

Preprocessing data for prediction...
Making predictions...
Predicting Team_score...
Predicting Opponent_score...
Predictions completed.

Preprocessing data for prediction...
Making predictions...
Predicting Team_score...
Predicting Opponent_score...
Predictions completed.



In [18]:
display(dfFinalPrediction1)
display(dfFinalPrediction2)

,Team,Opponent,Team_score_base,Opponent_score_base,Team_score_ensemble,Opponent_score_ensemble
0,Alabama,Kent St.,76.319274,49.136176,76.449941,50.147869
1,Colgate,Iona,72.954375,76.554651,72.670941,77.091359
2,Illinois,Missouri,76.505342,75.109084,76.283729,75.329047
3,Illinois St.,Cornell,88.216515,78.465517,88.345245,79.036661
4,Iowa St.,Morgan St.,92.303558,71.533991,92.826763,72.617681
5,Michigan,Purdue Fort Wayne,88.898072,56.060486,89.326932,57.351476
6,New Hampshire,Stonehill,83.875446,85.945578,83.253781,85.980964
7,Oklahoma,Central Arkansas,79.139136,61.754576,78.947988,62.252767


,Team,Opponent,Team_score_ensembleRF,Opponent_score_ensembleRF,Team_score_fineTuned,Opponent_score_fineTuned
0,Alabama,Kent St.,76.163815,48.229332,76.204800,49.534079
1,Colgate,Iona,72.707557,76.616127,72.786425,76.810909
2,Illinois,Missouri,76.506734,75.496050,76.409269,75.220219
3,Illinois St.,Cornell,88.230593,78.663784,88.227312,78.554133
4,Iowa St.,Morgan St.,92.281369,71.685553,92.257164,71.907440
5,Michigan,Purdue Fort Wayne,88.801091,55.423271,88.833063,56.567473
6,New Hampshire,Stonehill,83.880932,85.576636,83.779101,85.894041
7,Oklahoma,Central Arkansas,78.962362,62.552852,78.987219,61.972307


# Backtesting Models

In [19]:
# Train and evaluate Bayesian Ridge regression models for multiple target variables.
def baseModel(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    for target in y_train.columns:
        print(f"Training model for {target}...")

        # Initialize the model
        model = BayesianRidge()

        # Train the model
        model.fit(X_train, y_train[target])
        models[target] = model

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

        # Optional: Uncomment to print predictions
        # print("Actual:", y_test[target].values)
        # print("Predicted values:", y_pred)

    return models


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_main = baseModel(X_train, y_train, X_test, y_test)
print(model_main)

Training model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6670

Training model for Opponent_score...
Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.4352

{'Team_score': BayesianRidge(), 'Opponent_score': BayesianRidge()}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [20]:
def EnsembleModel(X_train, y_train, X_test, y_test, threshold=6):
    """
    Train and evaluate a StackingRegressor for multiple target variables.

    Parameters:
    X_train (DataFrame): Training feature set.
    y_train (DataFrame): Training target set (multi-target).
    X_test (DataFrame): Test feature set.
    y_test (DataFrame): Test target set (multi-target).
    threshold (int, optional): Threshold for accuracy evaluation. Default is 6.

    Returns:
    dict: A dictionary containing trained ensemble models for each target variable.
    """
    models = {}
    for target in y_train.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('ridge', Ridge()),
            ('bayesian_ridge', BayesianRidge())
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X_train, y_train[target])
        models[target] = stacking_model

        # Make predictions
        y_pred = stacking_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

        # Optional: Uncomment to print predictions
        # print("Actual:", y_test[target].values)
        # print("Predicted values:", y_pred)

    return models

# Example Usage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = EnsembleModel(X_train, y_train, X_test, y_test)
print(model2)


Training StackingRegressor model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6741

Training StackingRegressor model for Opponent_score...
Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.3860

{'Team_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge()), 'Opponent_score': StackingRegressor(estimators=[('ridge', Ridge()),
                              ('bayesian_ridge', BayesianRidge())],
                  final_estimator=BayesianRidge())}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [21]:
#Train and evaluate a StackingRegressor combining Bayesian Ridge and Random Forest for multiple target variables.

def EnsembleModelWithRF(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    for target in y_train.columns:
        print(f"Training StackingRegressor model for {target}...")

        # Define base models and stacking regressor
        base_models = [
            ('bayesian_ridge', BayesianRidge()),
            ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))
        ]
        stacking_model = StackingRegressor(estimators=base_models, final_estimator=BayesianRidge())

        # Train the stacking model
        stacking_model.fit(X_train, y_train[target])
        models[target] = stacking_model

        # Make predictions
        y_pred = stacking_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

    return models

# Example Usage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model3 = EnsembleModelWithRF(X_train, y_train, X_test, y_test)
print(model3)


Training StackingRegressor model for Team_score...
Accuracy for Team_score within 6 points: 67.21%
RMSE for Team_score: 5.6690

Training StackingRegressor model for Opponent_score...


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Accuracy for Opponent_score within 6 points: 68.85%
RMSE for Opponent_score: 5.3803

{'Team_score': StackingRegressor(estimators=[('bayesian_ridge', BayesianRidge()),
                              ('random_forest',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=BayesianRidge()), 'Opponent_score': StackingRegressor(estimators=[('bayesian_ridge', BayesianRidge()),
                              ('random_forest',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=BayesianRidge())}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
# Train and evaluate Bayesian Ridge regression models for multiple target variables
def baseModelWithHyperparameterTuning(X_train, y_train, X_test, y_test, threshold=6):
    models = {}
    
    for target in y_train.columns:
        print(f"Training model for {target}...\n")

        # Step 1: Broad parameter ranges for RandomizedSearchCV
        random_param_grid = {
            'alpha_1': np.logspace(-6, -2, 20),  # Wide range
            'alpha_2': np.logspace(-6, -2, 20),  # Wide range
            'lambda_1': np.logspace(-6, -2, 20), # Wide range
            'lambda_2': np.logspace(-6, -2, 20), # Wide range
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }
        
        model = BayesianRidge()
        scorer = 'neg_mean_squared_error'
        
        random_search = RandomizedSearchCV(
            model, random_param_grid, scoring=scorer, cv=5, n_iter=50, verbose=1, random_state=42
        )
        random_search.fit(X_train, y_train[target])
        best_params_random = random_search.best_params_
        print(f"Best parameters from RandomizedSearchCV for {target}:", best_params_random)
        
        # Step 2: Define a narrower grid around the best parameters
        grid_param_grid = {
            'alpha_1': np.linspace(
                best_params_random['alpha_1'] / 10, best_params_random['alpha_1'] * 10, 10
            ),
            'alpha_2': np.linspace(
                best_params_random['alpha_2'] / 10, best_params_random['alpha_2'] * 10, 10
            ),
            'lambda_1': np.linspace(
                best_params_random['lambda_1'] / 10, best_params_random['lambda_1'] * 10, 10
            ),
            'lambda_2': np.linspace(
                best_params_random['lambda_2'] / 10, best_params_random['lambda_2'] * 10, 10
            ),
            'tol': [1e-4, 1e-3, 1e-2],
            'max_iter': [100, 300, 500]
        }
        
        grid_search = GridSearchCV(model, grid_param_grid, scoring=scorer, cv=5, verbose=1)
        grid_search.fit(X_train, y_train[target])
        best_params_grid = grid_search.best_params_
        print(f"Best parameters from GridSearchCV for {target}:", best_params_grid)
        
        # Step 3: Train the model with the best parameters
        final_model = BayesianRidge(**best_params_grid)
        final_model.fit(X_train, y_train[target])
        models[target] = final_model

        # Step 4: Make predictions
        y_pred = final_model.predict(X_test)

        # Evaluate accuracy within the threshold
        accuracy = (abs(y_pred - y_test[target]) <= threshold).mean() * 100
        print(f"Accuracy for {target} within {threshold} points: {accuracy:.2f}%")

        # Evaluate RMSE
        rmse = mean_squared_error(y_test[target], y_pred, squared=False)
        print(f"RMSE for {target}: {rmse:.4f}\n")

    return models

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the model
#model_main = baseModelWithHyperparameterTuning(X_train, y_train, X_test, y_test)
#print(model_main)
